In [1]:
import sys
print(sys.executable)

/home/saumya/.conda/envs/penny_skl/bin/python


In [2]:
#import gpaw
#import os
#print(os.path.dirname(gpaw.__file__))

In [1]:
import numpy as np
import psi4
import os

# -------------------------
# 1. Psi4 options
# -------------------------
psi4.set_options({
    'scf_type': 'df',         # density fitting for speed
    'reference': 'rhf',
    'basis': 'sto-3g',        # minimal basis for demo
    'geom_maxiter': 1,
    'print': 0                 # suppress Psi4 output
})

# -------------------------
# 2. Generate LiH geometries
# -------------------------
distances = np.linspace(1.4, 3.0, 2400)  # bond distances in Angstrom
positions_list = []
energies_list = []
forces_list = []

for d in distances:
    # Define molecule
    mol = psi4.geometry(f"""
    Li 0.0 0.0 0.0
    H  {d} 0.0 0.0
    """)
    
    # Compute energy and gradient (forces)
    e, wfn = psi4.energy('scf', molecule=mol, return_wfn=True)
    grad = psi4.gradient('scf', molecule=mol)  # in Hartree/Bohr

    # Convert Psi4 gradient to NumPy array and flip sign for forces
    # Psi4 gradient = dE/dR, so forces = -gradient
    forces = -np.array(grad)

    # Store
    positions = np.array(mol.geometry())  # in Bohr
    positions = positions * 0.529177  # Convert Bohr -> Angstrom

    positions_list.append(positions)
    energies_list.append(e)  # in Hartree
    forces_list.append(forces * 27.2114 / 0.529177)  # Hartree/Bohr -> eV/Å

# Convert to NumPy arrays
positions_array = np.array(positions_list)  # (N, 2, 3)
energies_array = np.array(energies_list)    # (N,)
forces_array = np.array(forces_list)        # (N, 2, 3)

# -------------------------
# 3. Save for PennyLane tutorial
# -------------------------
os.makedirs("eqnn_force_field_data", exist_ok=True)
np.save("eqnn_force_field_data/Positions.npy", positions_array)
np.save("eqnn_force_field_data/Energy.npy", energies_array)
np.save("eqnn_force_field_data/Forces.npy", forces_array)

print("✅ LiH dataset generated with Psi4.")
print("   Positions:", positions_array.shape)
print("   Energy:   ", energies_array.shape)
print("   Forces:   ", forces_array.shape)



Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on saumya-ThiP
*** at Thu Nov 13 23:09:06 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry LI         line    31 file /home/saumya/.conda/envs/penny_skl/share/psi4/basis/sto-3g.gbs 
    atoms 2 entry H          line    19 file /home/saumya/.conda/envs/penny_skl/share/psi4/basis/sto-3g.gbs 

   => Loading Basis Set <=

    Name: (STO-3G AUX)
    Role: JKFIT
    Keyword: DF_BASIS_SCF
    atoms 1 entry LI         line    54 file /home/saumya/.conda/envs/penny_skl/share/psi4/basis/def2-universal-jkfit.gbs 
    atoms 2 entry H          line    18 file /home/saumya/.conda/envs/penny_skl/share/psi4/basis/def2-universal-jkfit.gbs 

  DFHelper Memory: AOs need 0.000 GiB; user suppl

KeyboardInterrupt: 

In [3]:
import numpy as np
import psi4
import os
from sklearn.preprocessing import MinMaxScaler

# -------------------------
# 1. Psi4 options
# -------------------------
psi4.set_options({
    'scf_type': 'df',         # density fitting for speed
    'reference': 'rhf',
    'basis': 'sto-3g',        # minimal basis for demo
    'geom_maxiter': 1,
    'print': 0                 # suppress Psi4 output
})

# -------------------------
# 2. Generate LiH geometries
# -------------------------
distances = np.linspace(1.4, 3.0, 2400)  # bond distances in Å
positions_list = []
energies_list = []
forces_list = []

for d in distances:
    # Define molecule
    mol = psi4.geometry(f"""
    Li 0.0 0.0 0.0
    H  {d} 0.0 0.0
    """)
    
    # Compute energy and gradient
    e, wfn = psi4.energy('scf', molecule=mol, return_wfn=True)
    grad = psi4.gradient('scf', molecule=mol)  # Hartree/Bohr

    # Convert Psi4 gradient to forces in eV/Å
    forces = -np.array(grad) * 27.2114 / 0.529177  # Hartree/Bohr -> eV/Å

    # Get positions in Angstrom
    positions = np.array(mol.geometry()) * 0.529177  # Bohr -> Å

    # Append data
    positions_list.append(positions)
    energies_list.append(e)   # Hartree
    forces_list.append(forces)

# Convert to NumPy arrays
positions_array = np.array(positions_list)  # (N, 2, 3)
energies_array = np.array(energies_list)    # (N,)
forces_array = np.array(forces_list)        # (N, 2, 3)

# -------------------------
# 3. Preprocessing (for PennyLane tutorial)
# -------------------------

# 3a. Scale energy to [-1, 1]
energy_2d = energies_array.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
energy_scaled_2d = scaler.fit_transform(energy_2d)
energy_scaled = energy_scaled_2d.flatten()

# 3b. Scale forces with same factor
forces_scaled = forces_array * scaler.scale_[0]

# 3c. Center molecule: place first atom (Li) at origin
positions_centered = positions_array - positions_array[:, 0:1, :]

# -------------------------
# 4. Save dataset
# -------------------------
os.makedirs("eqnn_force_field_data", exist_ok=True)

np.save("eqnn_force_field_data/Positions.npy", positions_centered)
np.save("eqnn_force_field_data/Forces.npy", forces_scaled)
np.save("eqnn_force_field_data/Energy.npy", energy_scaled)

print("✅ LiH dataset with preprocessing generated successfully!")
print("Positions:", positions_centered.shape)
print("Forces:", forces_scaled.shape)
print("Energy:", energy_scaled.shape)



Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on saumya-ThiP
*** at Thu Nov 13 23:35:40 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry LI         line    31 file /home/saumya/.conda/envs/penny_skl/share/psi4/basis/sto-3g.gbs 
    atoms 2 entry H          line    19 file /home/saumya/.conda/envs/penny_skl/share/psi4/basis/sto-3g.gbs 

   => Loading Basis Set <=

    Name: (STO-3G AUX)
    Role: JKFIT
    Keyword: DF_BASIS_SCF
    atoms 1 entry LI         line    54 file /home/saumya/.conda/envs/penny_skl/share/psi4/basis/def2-universal-jkfit.gbs 
    atoms 2 entry H          line    18 file /home/saumya/.conda/envs/penny_skl/share/psi4/basis/def2-universal-jkfit.gbs 

  DFHelper Memory: AOs need 0.000 GiB; user suppl

KeyboardInterrupt: 